In [7]:
import pandas as pd

file_path = "Underlying Cause of Death, 2018-2023, Single Race.csv"

# Try Latin-1 encoding instead of UTF-8
with open(file_path, "r", encoding="latin-1") as f:
    lines = f.readlines()

# Find header row with "Year" and "Deaths"
header_line = None
for i, line in enumerate(lines):
    if "Year" in line and "Deaths" in line:
        header_line = i
        break

print("Header starts at line:", header_line)



Header starts at line: 0


In [8]:
import pandas as pd

df = pd.read_csv(file_path, skiprows=header_line, encoding="latin-1")

# Drop unused columns
df = df.drop(columns=["Notes"], errors="ignore")

# Keep rows with valid death counts
df = df[df["Deaths"].notna()]

# Convert numeric columns
for col in ["Deaths", "Population", "Crude Rate"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

df.head()


,Year,Year Code,State,State Code,Cause of death,Cause of death Code,Deaths,Population,Crude Rate
0,2018.0,2018.0,New York,36.0,Enterocolitis due to Clostridium difficile,A04.7,273.0,19542209.0,1.4
1,2018.0,2018.0,New York,36.0,Other and unspecified gastroenteritis and coli...,A09.0,32.0,19542209.0,0.2
2,2018.0,2018.0,New York,36.0,Gastroenteritis and colitis of unspecified origin,A09.9,157.0,19542209.0,0.8
3,2018.0,2018.0,New York,36.0,"Tuberculosis of lung, without mention of bacte...",A16.2,14.0,19542209.0,NaN
4,2018.0,2018.0,New York,36.0,"Respiratory tuberculosis unspecified, without ...",A16.9,10.0,19542209.0,NaN


In [9]:
df.to_csv("ny_mortality_by_year.csv", index=False, encoding="utf-8-sig")

from google.colab import files
files.download("ny_mortality_by_year.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
import pandas as pd
import sqlite3


In [11]:

file_path = "ny_mortality_by_year.csv"

# Load into pandas
df = pd.read_csv(file_path)

# Preview data
df.head()


,Year,Year Code,State,State Code,Cause of death,Cause of death Code,Deaths,Population,Crude Rate
0,2018.0,2018.0,New York,36.0,Enterocolitis due to Clostridium difficile,A04.7,273.0,19542209.0,1.4
1,2018.0,2018.0,New York,36.0,Other and unspecified gastroenteritis and coli...,A09.0,32.0,19542209.0,0.2
2,2018.0,2018.0,New York,36.0,Gastroenteritis and colitis of unspecified origin,A09.9,157.0,19542209.0,0.8
3,2018.0,2018.0,New York,36.0,"Tuberculosis of lung, without mention of bacte...",A16.2,14.0,19542209.0,NaN
4,2018.0,2018.0,New York,36.0,"Respiratory tuberculosis unspecified, without ...",A16.9,10.0,19542209.0,NaN


In [12]:
# Create in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# Load pandas DataFrame into SQLite table
df.to_sql("mortality", conn, index=False, if_exists="replace")


4215

In [13]:
query = "SELECT * FROM mortality LIMIT 10;"
pd.read_sql(query, conn)


,Year,Year Code,State,State Code,Cause of death,Cause of death Code,Deaths,Population,Crude Rate
0,2018.0,2018.0,New York,36.0,Enterocolitis due to Clostridium difficile,A04.7,273.0,19542209.0,1.4
1,2018.0,2018.0,New York,36.0,Other and unspecified gastroenteritis and coli...,A09.0,32.0,19542209.0,0.2
2,2018.0,2018.0,New York,36.0,Gastroenteritis and colitis of unspecified origin,A09.9,157.0,19542209.0,0.8
3,2018.0,2018.0,New York,36.0,"Tuberculosis of lung, without mention of bacte...",A16.2,14.0,19542209.0,NaN
4,2018.0,2018.0,New York,36.0,"Respiratory tuberculosis unspecified, without ...",A16.9,10.0,19542209.0,NaN
5,2018.0,2018.0,New York,36.0,Tuberculosis of bones and joints,A18.0,12.0,19542209.0,NaN
6,2018.0,2018.0,New York,36.0,Pulmonary mycobacterial infection,A31.0,17.0,19542209.0,NaN
7,2018.0,2018.0,New York,36.0,"Streptococcal septicaemia, unspecified",A40.9,12.0,19542209.0,NaN
8,2018.0,2018.0,New York,36.0,Septicaemia due to Staphylococcus aureus,A41.0,35.0,19542209.0,0.2
9,2018.0,2018.0,New York,36.0,Septicaemia due to unspecified staphylococcus,A41.2,11.0,19542209.0,NaN


In [15]:
query = "SELECT SUM(Deaths) AS total_deaths FROM mortality;"
pd.read_sql(query, conn)


,total_deaths
0,4115049.0


In [16]:
query = """
SELECT Year, SUM(Deaths) AS yearly_deaths, AVG([Crude Rate]) AS avg_rate
FROM mortality
GROUP BY Year
ORDER BY Year;
"""
pd.read_sql(query, conn)


,Year,yearly_deaths,avg_rate
0,NaN,1034376.0,880.900000
1,2018.0,467820.0,5.179303
2,2019.0,465475.0,4.959916
3,2020.0,606446.0,6.416667
4,2021.0,540231.0,5.450101
5,2022.0,517973.0,5.129608
6,2023.0,482728.0,4.831953


from matplotlib import pyplot as plt
_df_2['Year'].plot(kind='hist', bins=20, title='Year')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['yearly_deaths'].plot(kind='hist', bins=20, title='yearly_deaths')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_4['avg_rate'].plot(kind='hist', bins=20, title='avg_rate')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_5.plot(kind='scatter', x='Year', y='yearly_deaths', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['Year']
  ys = series['avg_rate']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_8.sort_values('Year', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('avg_rate')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['Year']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'Year'}, axis=1)
              .sort_values('Year', ascending=True))
  xs = counted['Year']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_9.sort_values('Year', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('Year')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_10['Year'].plot(kind='line', figsize=(8, 4), title='Year')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['yearly_deaths'].plot(kind='line', figsize=(8, 4), title='yearly_deaths')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_12['avg_rate'].plot(kind='line', figsize=(8, 4), title='avg_rate')
plt.gca().spines[['top', 'right']].set_visible(False)

In [17]:
query = """
SELECT Year, SUM(Deaths) AS yearly_deaths
FROM mortality
GROUP BY Year
ORDER BY yearly_deaths DESC
LIMIT 1;
"""
pd.read_sql(query, conn)


,Year,yearly_deaths
0,None,1034376.0


In [18]:
query = """
SELECT Year, [Crude Rate]
FROM mortality
ORDER BY Year;
"""
pd.read_sql(query, conn)


,Year,Crude Rate
0,NaN,880.9
1,2018.0,1.4
2,2018.0,0.2
3,2018.0,0.8
4,2018.0,NaN
...,...,...
4210,2023.0,0.3
4211,2023.0,0.1
4212,2023.0,NaN
4213,2023.0,828.6


In [19]:
conn.close()
